In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = torch.flatten(x, 1) # Flatten the images
        return self.linear(x) # No activation function, outputs logits directly

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform) #Load dataset


'''
Split dataset into training and validation
'''

train_size = int(0.8 * len(dataset))  # Determine size of training set (80% of total)
val_size = len(dataset) - train_size # The remaining for validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

model = LogisticRegression(3 * 32 * 32, 10)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

'''
Function to train the model
'''
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # Set the model to training mode
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X) # Make predictions
        y_one_hot = nn.functional.one_hot(y, num_classes=10).float()
        loss = loss_fn(pred, y_one_hot)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

'''
Function to evaluate the model
'''
def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad(): # Disable gradient computation
        for X, y in dataloader:
            pred = model(X) # Make predictions
            y_one_hot = nn.functional.one_hot(y, num_classes=10).float()
            val_loss += loss_fn(pred, y_one_hot).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    accuracy = correct / size
    print(f"Validation Error: \n Avg loss: {val_loss:>8f} \n Accuracy: {accuracy:>8f}")


epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    validate(val_loader, model, loss_fn)
print("Training is finished")

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64)

validate(test_loader, model, loss_fn) # test


100%|██████████| 170498071/170498071 [00:02<00:00, 84432790.24it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Epoch 1
-------------------------------
loss: 2.312574  [    0/40000]
loss: 1.740957  [ 6400/40000]
loss: 1.826658  [12800/40000]
loss: 2.163486  [19200/40000]
loss: 1.945163  [25600/40000]
loss: 1.801677  [32000/40000]
loss: 1.731437  [38400/40000]
Validation Error: 
 Avg loss: 1.824731 
 Accuracy: 0.373900
Epoch 2
-------------------------------
loss: 1.910544  [    0/40000]
loss: 1.757744  [ 6400/40000]
loss: 1.934564  [12800/40000]
loss: 1.990225  [19200/40000]
loss: 1.730829  [25600/40000]
loss: 1.787988  [32000/40000]
loss: 1.589436  [38400/40000]
Validation Error: 
 Avg loss: 1.824560 
 Accuracy: 0.375500
Epoch 3
-------------------------------
loss: 1.947538  [    0/40000]
loss: 1.615619  [ 6400/40000]
loss: 1.620033  [12800/40000]
loss: 1.909129  [19200/40000]
loss: 1.997968  [25600/40000]
loss: 1.823106  [32000/40000]
loss: 1.786598  [38400/40000]
Validation Error: 
 Avg loss: 1.814404 
 Accuracy: 0.375400
Epoch 4
-----------